In [1]:
import numpy as np
import pandas as pd


In [2]:
sleep_df=pd.read_csv('../data/sleepdata_2.csv',delimiter=";")
sleep_df.sample(n=5)

,Start,End,Sleep Quality,Regularity,Mood,Heart rate (bpm),Steps,Alarm mode,Air Pressure (Pa),City,...,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Window start,Window stop,Did snore,Snore time,Weather temperature (°C),Weather type,Notes
812,2021-11-08 21:51:56,2021-11-09 04:46:52,65%,78%,NaN,0,798,Normal,94.7,Nelson,...,24895.4,18505.6,3402.4,2021-11-09 04:45:00,2021-11-09 04:45:00,True,540.0,3.0,Fog,NaN
254,2020-03-14 00:10:36,2020-03-14 06:37:00,61%,86%,NaN,0,857,No alarm,NaN,Nelson,...,23184.3,17774.6,2395.7,NaN,NaN,True,0.0,-14.8,Snow,NaN
915,2022-03-23 22:30:25,2022-03-24 06:03:07,72%,86%,NaN,0,2521,Normal,94.5,Central Kootenay,...,27162.0,25079.6,543.2,2022-03-24 04:45:00,2022-03-24 04:45:00,True,1080.0,4.6,Fog,NaN
355,2020-06-28 22:28:33,2020-06-29 05:01:17,57%,81%,NaN,0,6141,Normal,NaN,NaN,...,23564.2,21129.2,471.3,2020-06-29 05:00:00,2020-06-29 05:00:00,True,245.0,8.6,Sunny,NaN
872,2022-01-21 23:00:38,2022-01-22 08:20:21,96%,67%,NaN,0,4168,No alarm,96.0,Central Kootenay,...,33582.6,29888.5,671.7,NaN,NaN,True,1218.8,1.2,Partly cloudy,NaN


In [3]:
n_rows=sleep_df.shape[0]
n_cols=sleep_df.shape[1]
n_rows, n_cols

(921, 21)

#TODO: GIVE OBSERVATION ABOUT MEANING OF EACH ROWS, COLUMNS

- After finding meaning of each rows, each columns in the data, we check whether data have duplicate rows

In [4]:
duplicated_rows = sleep_df[sleep_df.duplicated()]
len(duplicated_rows)


0

There is no duplicate row in data. We notice that there are many missing data (NaN) in the CSV file. We need to find how many percentage of missing data there are.

In [5]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Start                        921 non-null    object 
 1   End                          921 non-null    object 
 2   Sleep Quality                921 non-null    object 
 3   Regularity                   921 non-null    object 
 4   Mood                         0 non-null      float64
 5   Heart rate (bpm)             921 non-null    int64  
 6   Steps                        921 non-null    int64  
 7   Alarm mode                   921 non-null    object 
 8   Air Pressure (Pa)            492 non-null    float64
 9   City                         487 non-null    object 
 10  Movements per hour           921 non-null    float64
 11  Time in bed (seconds)        921 non-null    float64
 12  Time asleep (seconds)        921 non-null    float64
 13  Time before sleep (s

Now we will caculate missing percent of each column

In [6]:
missing_values = sleep_df.isnull().sum()

# Calculate missing ratio for each column
missing_ratio = (missing_values / len(sleep_df)) * 100
missing_ratio

Start                            0.000000
End                              0.000000
Sleep Quality                    0.000000
Regularity                       0.000000
Mood                           100.000000
Heart rate (bpm)                 0.000000
Steps                            0.000000
Alarm mode                       0.000000
Air Pressure (Pa)               46.579805
City                            47.122693
Movements per hour               0.000000
Time in bed (seconds)            0.000000
Time asleep (seconds)            0.000000
Time before sleep (seconds)      0.000000
Window start                     8.686211
Window stop                      8.686211
Did snore                        0.000000
Snore time                       0.000000
Weather temperature (°C)         0.000000
Weather type                     0.000000
Notes                           99.022801
dtype: float64

From observing data and the result, we will drop columns that have >40% data missing, 

In [7]:
columns_to_drop = ['Mood','Air Pressure (Pa)','City','Notes']
sleep_df=sleep_df.drop(columns_to_drop,axis=1)

Check if any columns have same value in every cells and drop that columns

In [8]:
for column in sleep_df.columns:
    if sleep_df[column].nunique() == 1:
        print("Are all values in '{}' the same".format(column))
        sleep_df=sleep_df.drop(column,axis=1)

Are all values in 'Heart rate (bpm)' the same


## CATEGORICAL COLUMNS

Convert columns 'Start', 'End', 'Window start', 'Window stop' to datetime

In [9]:
sleep_df['Start']=pd.to_datetime(sleep_df['Start'],format='%Y-%m-%d %H:%M:%S')
sleep_df['End']=pd.to_datetime(sleep_df['End'],format='%Y-%m-%d %H:%M:%S')
sleep_df['Window start']=pd.to_datetime(sleep_df['Window start'],format='%Y-%m-%d %H:%M:%S')
sleep_df['Window stop']=pd.to_datetime(sleep_df['Window stop'],format='%Y-%m-%d %H:%M:%S')


COUNT UNIQUE VALUE OF columns `Alarm mode`, `Did snore` and `Weather type`  

In [10]:
cols_to_count=['Alarm mode','Did snore','Weather type']
df = sleep_df[cols_to_count]
cat_cols_profile={}
# TODO: code step 1
def compute_cat_cols(col):
    # Count the occurrences of each attribute in the column
    unique_attributes = df[col.name].value_counts().to_dict()
    cat_cols_profile[col.name]= {"num_diff_vals": len(unique_attributes), "distribution": unique_attributes}
# Use the agg method with the custom aggregation function
df.agg(compute_cat_cols)
cat_cols_profile=pd.DataFrame(data=cat_cols_profile,index=["num_diff_vals", "distribution"])

cat_cols_profile

,Alarm mode,Did snore,Weather type
num_diff_vals,2,2,9
distribution,"{'Normal': 841, 'No alarm': 80}","{True: 913, False: 8}","{'No weather': 488, 'Sunny': 121, 'Cloudy': 75..."


Value of `Alarm mode`, `Did snore` and `Weather` type seem normal

## NUMERIC COLUMNS

We see that columns `Sleep Quality` and `Regularity` contains percentage values. I think that we should change it to numeric format. Currently, they are stored as object type

In [11]:
sleep_df['Sleep Quality'] = sleep_df['Sleep Quality'].str.rstrip('%').astype('float')
sleep_df.rename(columns={'Sleep Quality': 'Sleep Quality (%)'}, inplace=True)

sleep_df['Regularity'] = sleep_df['Regularity'].str.rstrip('%').astype('float') 
sleep_df.rename(columns={'Regularity': 'Regularity (%)'}, inplace=True)

In [12]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Start                        921 non-null    datetime64[ns]
 1   End                          921 non-null    datetime64[ns]
 2   Sleep Quality (%)            921 non-null    float64       
 3   Regularity (%)               921 non-null    float64       
 4   Steps                        921 non-null    int64         
 5   Alarm mode                   921 non-null    object        
 6   Movements per hour           921 non-null    float64       
 7   Time in bed (seconds)        921 non-null    float64       
 8   Time asleep (seconds)        921 non-null    float64       
 9   Time before sleep (seconds)  921 non-null    float64       
 10  Window start                 841 non-null    datetime64[ns]
 11  Window stop                  841 non-null    

Caculate quantile of numeric columns

In [13]:
numeric_cols=['Sleep Quality (%)', 'Regularity (%)', 'Steps', 'Movements per hour', 'Time in bed (seconds)', 'Time asleep (seconds)', 'Time before sleep (seconds)', 'Snore time', 'Weather temperature (°C)']
numeric_col_profile = sleep_df[numeric_cols].describe().round(1)[1:]
numeric_col_profile

,Sleep Quality (%),Regularity (%),Steps,Movements per hour,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Snore time,Weather temperature (°C)
mean,77.9,84.4,5311.6,174.3,27505.5,23912.7,1001.6,312.9,2.1
std,12.9,12.5,4396.8,822.8,3916.7,4066.2,1046.3,455.5,6.3
min,7.0,-1.0,0.0,0.0,1852.8,0.0,0.0,0.0,-19.0
25%,71.0,82.0,2299.0,37.0,25500.6,21627.5,449.9,0.0,0.0
50%,78.0,86.0,4245.0,49.6,27422.9,23919.0,563.7,148.0,0.0
75%,86.0,91.0,7452.0,65.2,29530.1,26257.0,1287.0,440.0,3.1
max,100.0,100.0,38165.0,17926.7,46703.4,45769.4,5677.7,4477.3,24.5


In [14]:
sleep_df.to_csv('../data/Processed.csv')